El código extrae datos de reflectancia de imágenes Sentinel-2 para diferentes coordenadas en Estados Unidos utilizando Google Earth Engine. Luego, los datos se guardan en un archivo CSV.

<a target="_blank" href="https://colab.research.google.com/github/esjoal/ML_LAI_Estimation/blob/main/src/result_notebooks/collect_dataset_sat.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Importante:** Favor incluir tu project ID de Google Earth Engine editando el siguiente string.

In [21]:
project_ID = 'project-ml-jose'

In [22]:
import numpy as np
import pandas as pd
import ee

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
drive_path = '/content/drive/MyDrive/Colab Notebooks/'

Inicializamos la API

In [25]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project=project_ID)

Importamos las imagenes Sentinel-2 de reflectancia

In [26]:
# Cargamos a una variable la colección de imágenes Sentinel-2 MSI: Level-2A (Surface Reflectance) collection.
sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') # Para BOA reflectance
#sr = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') # Para TOA reflectance


Vemos los nombres de las bandas de las imagenes

In [27]:
sr.first().bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'AOT',
 'WVP',
 'SCL',
 'TCI_R',
 'TCI_G',
 'TCI_B',
 'MSK_CLDPRB',
 'MSK_SNWPRB',
 'QA10',
 'QA20',
 'QA60',
 'MSK_CLASSI_OPAQUE',
 'MSK_CLASSI_CIRRUS',
 'MSK_CLASSI_SNOW_ICE']

Definimos las coordenadas y los nombres de las zonas donde extraeremos la data de la imagen

In [28]:
coords = (('Bartlett Experimental Forest', 44.063901, -71.287308),
 ('Blandy Experimental Farm', 39.0602607727051, -78.0716400146484),
 ('Lyndon B. Johnson National Grassland', 33.40123, -97.57),
 ('Dead Lake', 32.54172, -87.80389),
 ('Central Plains Experimental Range', 40.81555, -104.74566),
 ('Disney Wilderness Preserve', 28.1250400543213, -81.43625),
 ('Harvard Forest', 42.5377998352051, -72.171501159668),
 ('Jones Ecological Research Center', 31.1948394775391, -84.468777),
 ('Jornada', 32.5907, -106.84261),
 ('Konza Prairie Biological Station', 39.110446, -96.612935),
 ('Lenoir Landing', 31.853861, -88.161181),
 ('Lajas Experimental Station', 18.02125, -67.0769),
 ('Moab', 38.24836, -109.38831),
 ('Niwot Ridge Mountain Research Station', 40.0543, -105.58245),
 ('Onaqui Ault', 40.1775894165039, -112.452438354492),
 ('Oak Ridge', 35.9641189575195, -84.282600402832),
 ('Ordway Swisher Biological Station', 29.6839, -81.9934),
 ('Pu u Maka ala Natural Area Reserve', 19.55309, -155.31731),
 ('Smithsonian Conservation Biology Institute', 38.8929214477539, -78.1395034790039),
 ('Smithsonian Environmental Research Center', 38.89016, -76.5601),
 ('Soaproot Saddle', 37.03337, -119.26219),
 ('Santa Rita', 31.91068, -110.83549),
 ('Steigerwaldt Land Services', 45.5089416503906, -89.5863723754883),
 ('North Sterling', 40.461952, -103.02934),
 ('Talladega National Forest', 32.9504585266113, -87.3932723999023),
 ('Lower Teakettle', 37.00583, -119.00602),
 ('University of Kansas Field Site', 39.040431, -95.19215),
 ('Woodworth', 47.128231048584, -99.2413635253906),
 ('Underc', 46.233959, -89.53751))

Filtramos las imagenes en el periodo de tiempo que nos interesa. Y seleccionamos las bandas de interés.

In [29]:
# Initial date of interest (inclusive).
#i_date = '2017-03-28'

# Final date of interest (exclusive).
#f_date = '2018-12-31'

df_lista = []

# Dividir las fechas en bloques de años para evitar errores de sobrecarga de memoria
years = ['2017','2018', '2019', '2020','2021','2022','2023','2024']  # Años de interés
for year in years:
    # Definir las fechas de inicio y fin para cada año
    if year == '2017':
        i_date = '2017-03-28'
    else:
        i_date = f'{year}-01-01'
    f_date = f'{year}-12-31'

    del sr # Liberamos la variable para cargarle la colección con otro rango de fechas

    # Selection of appropriate bands and dates for LST.
    sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    sr = sr.select('B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'SCL', 'QA10', 'QA20', 'QA60',).filterDate(i_date, f_date)

    # Verificar si hay imágenes disponibles para este año
    if sr.size().getInfo() == 0:
        print(f"No hay imágenes para el año {year}. Saltando...")
        continue  # Continuar con el siguiente año si no hay imágenes

    for site, my_lat, my_lon in coords:
      # Definimos en coordenadas de latitud y longitud el punto de interes donde queremos extraer los espectros de reflectancia.
      my_poi = ee.Geometry.Point(my_lon, my_lat)

      scale = 20  # scale in meters

      # Get the data for the pixel intersecting the point
      sr_point = sr.getRegion(my_poi, scale).getInfo()
      #sr_point = sr_point*0.0001

      # Verificar si se obtuvieron datos para el sitio y año actual
      if len(sr_point) == 0:
          print(f"No hay datos para el sitio {site} en el año {year}. Saltando...")
          continue  # Saltar este sitio si no hay datos

      df_individual = pd.DataFrame(sr_point[1:], columns=sr_point[0])
      df_individual['Site_sat'] = site
      df_lista.append(df_individual)

      # Liberar memoria explícitamente
      del sr_point

#Combinar todos los DataFrames en uno solo
df_sat= pd.concat(df_lista, axis=0, ignore_index=True)

KeyboardInterrupt: 

In [ ]:
df_sat

Grabamos una copia del dataframe crudo en formato csv

In [ ]:
#df_sat_backup = df_sat.copy()
#df_sat.to_csv(drive_path + 'S2_SR_USA_2017_2024_SCL.csv')